Dario Bertazioli 

3rd, June 2019


# Assignment 4: The Flow Shop Problem

The objective of the assignment is to apply different resolution methods to a non linear combinatorial problem. The problem that we will deal with is the Flow shop scheduling problem in which we have n machines and m jobs. Each job must be processed in all the machines following the same order, that is the i-th operation of the job must be executed on the i-th machine.

Moreover, we assume that machines can not work on more than one process simultaneously and that for each job the operation time in each machine is different and known at the beginning. You can assume assume that the order in which jobs are processed is exactly the same for all the machines.

The objective of the problem is to find an ordering of the jobs that minimizes total job execution time (also called makespan) that corresponds to the time on which all jobs get processed.

Your task is to implement one or different local search techniques in order to find a sub-optimal solution on the biggest possible set of instances and compare your results against:

* the best known solution values

* the results obtained using an algorithm available in R/Python.

## Implementation

In [1]:
#importing required modules
import pandas as pd
import numpy as np
import time
import copy

## Target Function

To solve the FlowShop problem, it is necessary to define the target function, which every algorithm that will be applied will try to minimize.

In literature, this objective function is well known as "Makespan" function, and so it will be called in this work.

This function will be the most relevant one, with function calls from every algorithm multiple (actually a lot of) times. Because of this consideration, i decided to implement such a function in a compiled Cython cell, for a better performance.

I tested the difference between a normal  python function and a compiled-in-Cython one, and the result is clear: the Cythonic one returns the call in less then $8~ \mu$s, whereas the Pythonic one takes a time of the order of ~$10$ ms.

In [2]:
%load_ext cython

In [3]:
%%cython
from numpy import zeros
cpdef calc_makespan( int [:] perm,int [:,:] times): #,m_seq):
    cdef int job_count = len(perm)
    cdef int machine_count =len(times[0])
    #makespan = [[0] * (machine_count + 1) for _ in range(0, job_count + 1)] 
    cdef int[:,::1] makespan=zeros((job_count+1,machine_count+1), dtype='int32')
    #for i, job in enumerate(perm):
    for i in range(job_count):
        #makespan[i][0]=0
        for machine in range(machine_count):
            makespan[i + 1][machine + 1] = max(makespan[i][machine + 1], makespan[i + 1][machine]) + times[perm[i]][machine]#times[job][machine]
    
    return makespan[job_count][machine_count]

The function basically calculates the total time for execution of all the jobs.

In order to do this, it sums, up to an i-th job/j-th machine, the total total time passed until this job (considering the previous jobs on previous machines) and adding the maximum value between the time passed until it finishes processing on the previous machine and the time passed for the other job to finish on the current machine. 

This procedure is iterated up to the last machine.

Consequently, we load the datasets for defining the makespan function.

In [24]:
#requiring the particular time matrix to the user
no_of_jobs=int(input('Please input the number of jobs: ') or 100) # default value is 20
no_of_machines=float(input('Please input the number of machines: ') or 20) # default value is 0.8
print("required computations on jobs: {}, machines: {}".format(no_of_jobs, no_of_machines) )

if no_of_jobs==20 and no_of_machines==5:
    #first of taill. 20x5 matrices, converted in xls for easy/fast parsing
    df_tmp=pd.read_excel("/home/dario/Desktop/DecisionModel/Assignment_4/20x5_flowshop.xlsx",
                         sheet_name="S1",
                         index_col =[0])     upper_b=1278
elif no_of_jobs==100 and no_of_machines==20:
    #first of taill. 100x20 matrices
    df_tmp=pd.read_excel("/home/dario/Desktop/DecisionModel/Assignment_4/100x20_flowshop.xlsx",
                         sheet_name="S1",
                         index_col =[0]) 
    upper_b=6286
elif no_of_jobs==500 and no_of_machines==20:
    #first of taill. 500x20 matrices
    df_tmp=pd.read_excel("/home/dario/Desktop/DecisionModel/Assignment_4/500x20_flowshop.xlsx",
                         sheet_name="S1",
                         index_col =[0]) 
else:
    print("unsupported number of jobs or number of machines (not implemented)")

m=df_tmp.T.values #Transposing for personal preference and extracting the correspondent numpy 2D array
m=m.astype("int32") #ensuring it to be of type "int32", for better computational performance
print(m.shape)

#m_list=df_tmp.T.values.tolist() #for a easier iteration/slicing, 
#but slower on makespan access (not cythonizable)

Please input the number of jobs: 20
Please input the number of machines: 5
required computations on jobs: 20, machines: 5.0
(20, 5)


To give an idea of the Flow Shop let's take a look to the Gantt Chart of the problem, showing the disposition of the jobs over different machines, considering the white spacing between jobs as idle times. 

![Initial GAnt Chart](makespan_initial_20x5.png "Title")

## Algorithms

### Genetic Algorithm

As a first try, I like to start with the genetic algorithm, having very appreciated it during the explanatory lesson.

However, reading the regarding literature, i do not expect it to be particularly fast.
I tested it on a 20x5, 100x20 and 500x20 (it becomes quite slow on the last one).

In [4]:
%%cython

import pandas as pd
import numpy as np
import time
import copy
from numpy import zeros

cpdef calc_makespan( int [:] perm,int [:,:] times): #,m_seq):
    cdef int job_count = len(perm)
    cdef int machine_count =len(times[0])
    #makespan = [[0] * (machine_count + 1) for _ in range(0, job_count + 1)]
    cdef int[:,::1] makespan=zeros((job_count+1,machine_count+1), dtype='int32')
    #print(makespan)
    #for i, job in enumerate(perm):
    for i in range(job_count):
        #makespan[i][0]=0
        for machine in range(machine_count):
            makespan[i + 1][machine + 1] = max(makespan[i][machine + 1], makespan[i + 1][machine]) + times[perm[i]][machine]#times[job][machine]
    
    return makespan[job_count][machine_count]

cpdef GA(int [:,:] df, 
         int population_size=5000, 
         double crossover_rate=1.,
         double mutation_p=0.1,
         double mutation_gen_rate=0.2,
         int n_iteration=100
        ):
    
    cdef int job_count=len(df)
    cdef int n_mutations=round(job_count*mutation_gen_rate)
    
    #####################
    # create population #
    #####################
    
    best_makespan=1000000000
    best_list,best_obj=[],[]
    population=[]
    
    for i in range(population_size):
        
        nxm_random_num=list(np.random.permutation(job_count)) # generate a random permutation
        population.append(nxm_random_num) # insert in population

    for n in range(n_iteration):
        
        curr_best_makespan=1000000000           
        
        #############
        # crossover #
        #############
        
        parents=copy.deepcopy(population) #better use deecopy for dereferentiation/
        mut_list=copy.deepcopy(population)
        rand_seq=list(np.random.permutation(population_size)) # random sequence for the crossover

        for m in range(int(population_size/2)):
            
            crossover_trhs=np.random.rand()
            
            if crossover_rate>=crossover_trhs:
            
                p1= population[rand_seq[2*m]][:]
                p2= population[rand_seq[2*m+1]][:]
                first_child=['na' for i in range(job_count)]
                second_child=['na' for i in range(job_count)]
                c=round(job_count/2)
                swap_ind=list(np.random.choice(job_count, c, replace=False))

                for index in range(c):
                
                    first_child[swap_ind[index]]=p2[swap_ind[index]]
                    second_child[swap_ind[index]]=p1[swap_ind[index]]
                
                c1=[p1[i] for i in range(job_count) if p1[i] not in first_child]
                c2=[p2[i] for i in range(job_count) if p2[i] not in second_child]

                for i in range(job_count-c):
                
                    first_child[first_child.index('na')]=c1[i]
                    second_child[second_child.index('na')]=c2[i]
                
                mut_list[rand_seq[2*m]]=first_child[:]
                mut_list[rand_seq[2*m+1]]=second_child[:]
        
        ############
        # mutation #
        ############
        
        for m in range(len(mut_list)):
            
            mut_thrs=np.random.rand()
            
            if mutation_p >= mut_thrs:
                # chooses the mutation pos.
                swap_ind=list(np.random.choice(job_count, n_mutations, replace=False)) 
                tmp_swap=mut_list[m][swap_ind[0]] 
                
                for i in range(n_mutations-1):
                    mut_list[m][swap_ind[i]]=mut_list[m][swap_ind[i+1]] #shifting (might be ameliorated)

                mut_list[m][swap_ind[n_mutations-1]]=tmp_swap 

        #######################
        # fitness calculation #
        #######################
        
        gene_fin=copy.deepcopy(parents)+copy.deepcopy(mut_list) #parent and mutated
        gene_fitness,gene_fit=[],[]
        fitness=0
        mks=[0 for elem in range(2*population_size)]
        
        for c in range(2*population_size):
            mks[c]=calc_makespan(np.array(gene_fin[c], dtype="int32"),df)
            gene_fitness.append(1/mks[c])
            gene_fit.append(mks[c])
            fitness+=gene_fitness[c]
        
        #########################
        # individuals selection #
        #########################
        
        ind_fit_list,cum_fit_list=[],[]

        for i in range(population_size*2):
            ind_fit_list.append(gene_fitness[i]/fitness)
        for i in range(population_size*2):
            cum_sum=0
            for j in range(0,i+1):
                cum_sum=cum_sum+ind_fit_list[j]
            cum_fit_list.append(cum_sum)

        rand_sel=[np.random.rand() for i in range(population_size)]

        for i in range(population_size):
            if rand_sel[i]<=cum_fit_list[0]:
                population[i]=copy.deepcopy(gene_fin[0])
                break
            else:
                for j in range(0,population_size*2-1):
                    if rand_sel[i]>cum_fit_list[j] and rand_sel[i]<=cum_fit_list[j+1]:
                        population[i]=copy.deepcopy(gene_fin[j+1])
        
        ##########################
        # updating best makespan #
        ##########################
        
        for i in range(population_size*2):
            if gene_fit[i]<curr_best_makespan:
                curr_best_makespan=gene_fit[i]
                now_perm=copy.deepcopy(gene_fin[i])

        if curr_best_makespan<=best_makespan:
            best_makespan=curr_best_makespan
            best_perm=copy.deepcopy(now_perm)
    
    
    print("best permutation",best_perm)
    print("best makespan:%f"%best_makespan)
    return best_perm,best_makespan

Testing with the 20x5 matrix

In [12]:
GA(m, n_iteration=1)

best permutation [14, 8, 18, 5, 13, 2, 1, 0, 6, 7, 15, 3, 4, 17, 11, 16, 10, 12, 9, 19]
best makespan:1306.000000


([14, 8, 18, 5, 13, 2, 1, 0, 6, 7, 15, 3, 4, 17, 11, 16, 10, 12, 9, 19], 1306)

In [41]:
GA(m, n_iteration=10)

best permutation [0, 7, 16, 1, 5, 15, 8, 12, 13, 3, 2, 14, 18, 11, 4, 17, 10, 9, 6, 19]
best makespan:1322.000000


([0, 7, 16, 1, 5, 15, 8, 12, 13, 3, 2, 14, 18, 11, 4, 17, 10, 9, 6, 19], 1322)

As we anticipated, the performances seems not to be so great.

After reading some literature, I tried some parameters tuning (as indicated e.g. in "Khan, Bazul & Govindan, Kannan & Jeyapaul, R. (2010 IJAOM).

However, the results did not change so much, so that after comparing with the python library "pyeasyga", whose algorithm results were similar to those obtained previously, i tried out other algorithm.

In [27]:
import random
from pyeasyga import pyeasyga

# setup seed data
data=list(range(no_of_jobs))

# initialise the GA
ga = pyeasyga.GeneticAlgorithm(seq.tolist(),
                            population_size=200,
                            generations=100,
                            crossover_probability=0.8,
                            mutation_probability=0.1,
                            elitism=True,
                            maximise_fitness=True)

# define and set function to create a candidate solution representation
def create_individual(data):
    individual = data[:]
    random.shuffle(individual)
    return individual

ga.create_individual = create_individual

# define and set the GA's crossover operation
def crossover(parent_1, parent_2):
    crossover_index = random.randrange(1, len(parent_1))
    child_1a = parent_1[:crossover_index]
    child_1b = [i for i in parent_2 if i not in child_1a]
    child_1 = child_1a + child_1b

    child_2a = parent_2[crossover_index:]
    child_2b = [i for i in parent_1 if i not in child_2a]
    child_2 = child_2a + child_2b
    
    return child_1, child_2

ga.crossover_function = crossover

# define and set the GA's mutation operation
def mutate(individual):
    mutate_index1 = random.randrange(len(individual))
    mutate_index2 = random.randrange(len(individual))
    individual[mutate_index1], individual[mutate_index2] = individual[mutate_index2], individual[mutate_index1]

ga.mutate_function = mutate

# define and set the GA's selection operation
def selection(population):
    #print(population)
    
    return random.choice(population)

ga.selection_function = selection

# define a fitness function
def fitness (individual, data):
    fitness=0
    v=calc_makespan(np.array(individual, dtype="int32"), m)
    fitness+= 1/v
    return fitness
ga.fitness_function = fitness       # set the GA's fitness function

In [25]:
ga.run()                            # run the GA

In [30]:
fit, sel_perm = ga.best_individual()
calc_makespan(np.array(sel_perm, dtype="int32"), m), sel_perm

(1297, [16, 14, 2, 18, 12, 10, 13, 15, 0, 17, 8, 5, 6, 1, 3, 4, 7, 11, 9, 19])

Which is not an horrible result, but it can be explained by the few iterations done (not having some great computational power), and probably by the not-optimal choice of the algorithm parameters  

## Simulated Annealing (SA)

The second algorithm I would like to implement is the simulated annealing algorithm.

The basic idea is to allow exploration of different configurations in order to find a candidate to the optimal sequence. 

The initial temperature, a parameter controlling the possibility to explore different configurations (higher T means higher probability to explore states characterized by higher "energy", in this case higher makespan total time), is slowly decreased to allow the convergence to a good final sequence, hopefully characterized by a low value of the makespan function.  

Firstly let us implement the *swap move function*, which will allow the exploration of the landscape of possible sequence configuration.  

In [247]:
def swapJobs (seq_init,n_swap=2, copy=True):#, verbose=False):
    if copy:
        seq=np.copy(seq_init)
    else:
        seq=seq_init #usefull for the library annealer (being a void function basically)
        
    index1=random.randint(0,len(seq)-1, size=n_swap)
    index2=random.randint(0,len(seq)-1, size=n_swap)
    
    #if verbose: #commenting for fasting up
    #    print("index: ",index1, "  ",index2)
    for i in range(n_swap):
        seq[[index1[i], index2[i]]] = seq[[index2[i], index1[i]]]
    return seq

In [187]:
from math import exp
def SimAnn(seq_init, m, tmax=5, tmin=0.01, alpha=0.9999, t_ini=10, n_swap=2, maxIterNoChange=200, scale_t=True, n_iterations=1000):
    
    seq=np.copy(seq_init)
    Temp = t_ini
    
    best_mks = calc_makespan(seq, m)
    now_mks=best_mks
    iterNoChange = 0
    maxit = n_iterations
    n_it = 0
    
    while(Temp >= tmin and n_it <= maxit):
        iterNoChange += 1
        n_it += 1
        new_seq = swapJobs(seq, n_swap = n_swap)
        curr_mks = calc_makespan(new_seq, m)
    
        if curr_mks <= best_mks:    
            seq = np.copy(new_seq)
            now_mks = curr_mks
            best_mks = now_mks
            iterNoChange = 0
            
        elif exp((now_mks-curr_mks)/Temp) > np.random.uniform(0, 1, 1):
            now_mks = curr_mks
            seq = np.copy(new_seq)
            iterNoChange = 0
          
        Temp = Temp*alpha   #linear decrease of temperature, 
                            #I also tried exp and log but did not notice great differences

        if iterNoChange >= maxIterNoChange:
            print("Reached max number of iteration without improvement( {}) after {} iterations".format(iterNoChange, n_it))
            break
          
    if(Temp<=tmin):
        print("Reached temperature low limit in {} iterations".format(n_it))
    
    print("exiting Temp ", temp)
    return seq, best_mks

Let us test the algorithm on the 20x5 matrix:

In [198]:
#seq_init=np.array(range(no_of_jobs), dtype="int32") #fixed init
seq_init=np.array(list(np.random.permutation(no_of_jobs)), dtype="int32") #random seed permutation
%time SimAnn(seq_init, m, n_iterations=100000000, maxIterNoChange=500, t_ini=50,tmin=0.0001, n_swap=3)

Reached max number of iteration without improvment( 500) after 27310 iterations
3.2572603099406447
CPU times: user 704 ms, sys: 7.83 ms, total: 712 ms
Wall time: 708 ms


(array([ 8, 16, 14,  7,  2,  5, 10, 12, 13,  1,  0,  4,  6, 18,  3, 17, 15,
         9, 19, 11], dtype=int32), 1278)

It seems to perform quite well, obtaining a candidate for a low-makespan state

### SA library:  simanneal

In [244]:
import sys
try: 
    import simanneal
except ImportError:
    !{sys.executable} -m pip install simanneal --user  # from pypi
    import simanneal

In [272]:
from simanneal import Annealer

class FlowShopProb(Annealer):
    """Test annealer with a FlowShopProb."""
    def move(self):
        """perform a swap"""
        swapJobs(self.state, n_swap=3, copy=False)
        
    def energy(self):
        """Calculates the length of the route."""
        return calc_makespan(self.state, self.mks_matrix)
    def __init__(self, state, matrix):
        self.mks_matrix = matrix
        super(FlowShopProb, self).__init__(state) # important!


In [339]:
flow = FlowShopProb(seq, m)
flow.steps = 100000
flow.copy_strategy = "deepcopy"
flow.Tmin=0.01

%time state, e = flow.anneal()

state, e

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01000       1278.00     0.60%     0.00%     0:00:03     0:00:00

CPU times: user 2.65 s, sys: 24 ms, total: 2.68 s
Wall time: 2.64 s


(array([ 8, 14,  5, 13,  2, 16,  0,  4, 10,  6,  1,  7, 12, 18,  3, 17, 15,
         9, 19, 11], dtype=int32), 1278)

The library algorithm converges at the same result. 

However, it is a bit slower than my implementation.

This is reasonable, even because in the doc it is clearly stated that such a module does not aim to great performance, being entirely written in Python, but it aims to give a practical tool for a fast initial exploration of the solution landscape in a optimization problem.  

## NEH Heuristic

I implemented the NEH Heuristic, following the method explained in literature (e.g. Kalczynski,  Kamburowski, Omega 35 (2007) 53–60)

In [5]:
%%cython

import numpy as np
from numpy import random #faster for rng tuples
from random import randint  #faster for one shot rng

from numpy import zeros

cpdef calc_makespan( int [:] perm,int [:,:] times): #,m_seq):
    cdef int job_count = len(perm)
    cdef int machine_count =len(times[0])
    #makespan = [[0] * (machine_count + 1) for _ in range(0, job_count + 1)]
    cdef int[:,::1] makespan=zeros((job_count+1,machine_count+1), dtype='int32')
    #print(makespan)
    #for i, job in enumerate(perm):
    for i in range(job_count):
        #makespan[i][0]=0
        for machine in range(machine_count):
            makespan[i + 1][machine + 1] = max(makespan[i][machine + 1], makespan[i + 1][machine]) + times[perm[i]][machine]#times[job][machine]
    
    return makespan[job_count][machine_count]


cpdef NEH(int [:,:] m, verbose=False):
    
    sum_jobs=np.sum(m, axis=1)
    n_jobs=m.shape[0]
    
    sorted_init_perm = np.lexsort((np.array(range(n_jobs)),-sum_jobs)) # Sort by sum_jobs, then index in  start_seq
    sorted_init_perm_i = sorted_init_perm[:2] # init permutation sorted 
    n_positions= len(sorted_init_perm_i) #actually =2
    
    mk1=calc_makespan(sorted_init_perm_i.astype("int32"), m)#try the makespan in the init seq
    
    rev_sorted_init_perm_i=sorted_init_perm_i[::-1] #not really working in cython
    mk2=calc_makespan(rev_sorted_init_perm_i.astype("int32"), m)#try it in the reversed one

    if mk1 <= mk2 :   #save initial best_makespan and starting sequence
        best_makespan=mk1
        best_seq_init=sorted_init_perm_i
    
    else: 
        best_makespan=mk2
        best_seq_init=rev_sorted_init_perm_i

    n_positions=3 #starting points for the insert of NEH algorithm

    curr_perm=best_seq_init
    for new_job in sorted_init_perm[2:]:
        #print(i)
        best_makespan=9999999
        
        for position in range(n_positions):
            temp_perm=np.insert(curr_perm, position, new_job)
            temp_mks=calc_makespan(temp_perm.astype("int32"), m)
            
            if temp_mks < best_makespan:
                best_makespan=temp_mks
                best_seq=temp_perm
    
        curr_perm=best_seq        
        n_positions+=1
    
    if verbose:
        print("best seq: ", best_seq)
        print("best mks: ", best_makespan)

    return best_seq.astype("int32"), best_makespan

In [199]:
NEH_seq, mks=NEH(m)
NEH_seq,mks

(array([ 2, 16,  8,  7, 14, 13, 10, 15, 12, 18,  5,  3,  4, 17,  0,  1,  9,
         6, 19, 11], dtype=int32), 1286)

It produces a good permutation, with the correspondent makespan corresponding to the upper bound indicated for such a matrix

Typically, such a permutation is taken as a starting point for another algorithm, to increase its performances.

Therefore, I implemented another algorithm, called Iterated Local Search. But let us first test the NEH resulting sequence on the Simulated Annealing process.

In [241]:
%time SimAnn(NEH_seq, m, n_iterations=100000, alpha=0.999,maxIterNoChange=1000, t_ini=5,tmin=0.001, n_swap=3)

Reached temperature limit in 8513 iterations
0.0009999338538073665
CPU times: user 240 ms, sys: 31 µs, total: 240 ms
Wall time: 237 ms


(array([ 8,  2, 10, 14, 16,  5,  1, 12,  3, 18, 13,  4,  0, 17,  6,  7, 15,
         9, 19, 11], dtype=int32), 1278)

Indeed, exploiting the good starting point (thus choosing a much lower initial temperature not to allow the system to explore too far from of our initial solution, we reach the same results as before with a reduced number of iterations. 

The fact we obtained the same value of the candidate min makespan for such a matrix might be a hint of a pretty good solution. 

## Iterated Local Search

It consists in Iterating the local search heuristic procedure after perturbing the resulting sequence at each step, in such a way to check for more possibilities and better explore the phase space. 

In the implementation I followed references like the following:  "Applying Iterated Local Search to the Permutation Flow Shop Problem (Stutzle et al.)".

The perturb function apply a perturbation on a sequence, by swapping n-pairs of adjacent numbers and j-pairs of distant numbers.

(The max dist (interval) parameter is taken from the reference :"Local search methods for the flowshop scheduling problem with flowtime minimization (Quan Ke Pan, Rubén Ruiz)" )

In [6]:
def perturb(seq, n_swaps=2, n_changes=1):#, verbose=False):
    ###n_swaps: number of pair swaps
    ###n_changes: number of long range swaps (let's say in range ~30) cfr.#ADDCIT

    #short range swaps
    index=random.randint(0,len(seq)-1, size=n_swaps+n_changes)
    
    #if verbose:
    #    print("index: ",index)
    
    for i in range(n_swaps):
        seq[[index[i], index[i]+1]] = seq[[index[i]+1, index[i]]]
        
        #if verbose:
        #    print("swapping {}-th elem on the right".format(index[i]))
    
    #long range swaps
    interval = max(round(len(seq)/5), 30) #max long range swap window
                                        #30 is an empirical number suggested in [cit] #ADDCIT 
    if(n_changes==1):
    
        if (len(seq) > 60):
            delta_index = randint(1, interval-1)
        else:
            delta_index = randint(1, int(len(seq)/2))
        
        if index[2]+delta_index<len(seq):
                sign=1
        else:
                sign=-1
                
        seq[[index[2], index[2]+sign*delta_index]] = seq[[index[2]+sign*delta_index, index[2]]]
        #if verbose:
        #    print("exchanged {} with {}".format(index[2], index[2]+delta_index))
    
    else: 
        
        for i in range(n_changes):
          
            if (len(seq) > 60):
                delta_index = randint(1, interval-1)
            else:
                delta_index = randint(1, int(len(seq)/2))
            
            if index[i+n_swaps]+delta_index<len(seq):
                sign=1
            else:
                sign=-1
                
            seq[[index[i+n_swaps], index[i+n_swaps]+sign*delta_index]] = seq[[index[i+n_swaps]+sign*delta_index, index[i+n_swaps]]]

    return seq

The LocalSearch function performs a local search: given a permutation, the function rearranges the order in a sub-sequential way storing the optimal value obtained during the process

In [7]:
def LocalSearch(perm,m, debug=True):
    best_makespan = 999999 #calc_makespan(perm, m)
    perm_save=perm
    
    for i in range(len(perm_save)):
        elem=perm[i]
        t_perm=np.delete(perm, i)
        #if debug: 
        #    if len(np.unique(t_perm))!=(len(perm)-1): print("halp: ",len(np.unique(t_perm))) 
        
        for j in range(len(t_perm)):
            curr_perm=np.insert(t_perm, j, elem)
            curr_mks=calc_makespan(curr_perm, m)
            
            #if debug: 
            #    if len(np.unique(curr_perm))!=len(perm): print("halp1: ",len(np.unique(curr_perm))) 
        
            if curr_mks <= best_makespan:
                best_makespan=curr_mks
                best_perm=curr_perm
        
        perm=best_perm
        
    return best_perm, best_makespan#, calc_makespan(best_perm, m)

Finally, in the ILS procedure we start from a sequence, perform the local search, then in a loop we perturb the obtained sequence and re-perform the local search, always keeping the best result from such a process.

In [8]:
def IteratedLocalSearch(perm, m, maxIterations=10000, maxIterNoChange=2000, verbose=False):#, break_if_no_changes= False):
    
    best_perm, best_makespan=LocalSearch(perm, m)
    now_perm=np.copy(best_perm)
    
    count=0
    count_no_change=0
    
    while count < maxIterations:
        curr_perm, curr_mks = LocalSearch(perturb(now_perm), m)
    
        if curr_mks <= best_makespan:
            best_makespan = curr_mks
            best_perm = np.copy(curr_perm)
            now_perm = np.copy(curr_perm)
            count_no_change = 0
    
        count+=1
        count_no_change += 1
        
        if count_no_change == maxIterNoChange:
            print("no more changes in optimum val")
            print("n_it : ", count)
            break
        
    return best_perm, best_makespan

This algorithm seems to performing pretty well. Indeed, for the 20x5 matrix, given a sequence corresponding (just as an example) to the range of the jobs, it produces the following result:

In [47]:
newseq= np.array(list(range(no_of_jobs)), dtype="int32")
IteratedLocalSearch(newseq, m, verbose=True,maxIterations=10000, maxIterNoChange=2000)

no more changes in optimum val


(array([ 8, 14,  5, 13, 16,  7, 10, 12,  2, 15,  0,  4,  6, 17, 18,  3,  1,
         9, 19, 11], dtype=int32), 1278)

Producing an optimal value with 2000 iterations.
However, starting from the result of the NEH algorithm, the performance can be improved:

In [91]:
%time IteratedLocalSearch(seq, m, verbose=True,maxIterations=200, maxIterNoChange=200)

no more changes in optimum val
CPU times: user 3.3 s, sys: 5 µs, total: 3.3 s
Wall time: 3.29 s


(array([ 8, 14, 16,  7,  2,  0, 18, 13,  5,  6, 10,  4,  3,  1, 12, 17, 15,
         9, 19, 11], dtype=int32), 1278)

Reaching the optimal values in 200 iterations

Let's take a look to the Gantt representation of the optimal solution that we found, comparing such a chart with the previous. It can be noticed a better disposition minimizing idle times.

![Gantt chart](makespan_initial_20x5.png "Gantt chart of the initial configuration") ![Gantt chart](makespan_final_20x5.png "Gantt chart of the final configuration")

## Higher rank matrices

Test of the implemented algorithm on the other matrices

### 100 jobs and 20 machines 

the considered matrix is as usual the first in the related page on Taillard website.

In [283]:
#requiring the particular time matrix to the user
no_of_jobs=int(input('Please input the number of jobs: ') or 100) # default value is 20
no_of_machines=float(input('Please input the number of machines: ') or 20) # default value is 0.8
print("required computations on jobs: {}, machines: {}".format(no_of_jobs, no_of_machines) )

if no_of_jobs==20 and no_of_machines==5:
    df_tmp=pd.read_excel("/home/dario/Desktop/DecisionModel/Assignment_4/20x5_flowshop.xlsx",sheet_name="S1",index_col =[0]) #first of taill. 20x5 matrices
    upper_b=1278
elif no_of_jobs==100 and no_of_machines==20:
    df_tmp=pd.read_excel("/home/dario/Desktop/DecisionModel/Assignment_4/100x20_flowshop.xlsx",sheet_name="S1",index_col =[0]) #first of taill. 100x20 matrices
    upper_b=6286
elif no_of_jobs==500 and no_of_machines==20:
    df_tmp=pd.read_excel("/home/dario/Desktop/DecisionModel/Assignment_4/500x20_flowshop.xlsx",sheet_name="S1",index_col =[0]) #first of taill. 500x20 matrices
else:
    print("unsupported number of jobs or number of machines (not implemented)")

#print(pt_tmp.head())
m1=df_tmp.T.values #Transposing for personal preference and extracting the correspondent numpy 2D array
m1=m1.astype("int32") #ensuring it to be of type "int32", for better computational performance
print(m1.shape)

#m_list=df_tmp.T.values.tolist() #for a better iteration
#print(m_list)

Please input the number of jobs: 100
Please input the number of machines: 20
required computations on jobs: 100, machines: 20.0
(100, 20)


#### NEH solution

In [307]:
%time seq1, mks1=NEH(m1)
seq1,mks1

In [61]:
upper_bound_100x20=6286

(mks1-upper_bound_100x20)*100/upper_bound_100x20

4.05663378937321

the NEH solution is 4% far from the upper-bound.

Trying to improve the performance with the iterated local search, we obtain:

In [52]:
IteratedLocalSearch(seq1, m1, verbose=True,maxIterations=300, maxIterNoChange=200)

no more changes in optimum val
n_it :  500
CPU times: user 8min 13s, sys: 24 ms, total: 8min 13s
Wall time: 8min 14s


(array([21, 53, 46, 73, 10, 50, 76, 30, 39, 23, 89, 82, 54, 49, 77, 64, 32,
        38, 79, 58, 99, 29, 15, 90, 26,  9, 19, 45, 11, 20,  2, 43, 13, 27,
        92, 31, 98, 60,  0,  8, 62,  1, 33, 36, 95, 66, 94, 65, 69, 55,  3,
         7, 14,  6, 34, 85, 80, 44,  5, 18, 52, 84, 24, 22, 59, 81, 87, 16,
         4, 75, 71, 97, 78, 47, 83, 70, 88, 72, 91, 63, 28, 67, 56, 86, 61,
        42, 57, 74, 17, 35, 93, 25, 51, 96, 41, 48, 37, 68, 40, 12],
       dtype=int32), 6516)

In [62]:
(6516-upper_bound_100x20)*100/upper_bound_100x20

3.658924594336621

the ILS solution is 3.7% far from the upper-bound 

The result is not far from the indicated upper bound. However, I could reach a pretty better sequence just my adding more iterations, with the counterpart of a greater computation time

#### SA on 100x20

Again, starting from the NEH resulting sequence, we try to optimize the makespan via SA:

In [308]:
%time SimAnn(seq1, m1, n_iterations=1000000, alpha=0.999995,maxIterNoChange=100000, t_ini=7,tmin=0.001, n_swap=3)

Reached max number of iteration without improvment( 100000) after 597766 iterations
0.3524215188550217
CPU times: user 58.8 s, sys: 1.25 ms, total: 58.8 s
Wall time: 58.8 s


(array([21, 53, 54, 10, 73, 78, 30, 81, 32, 89, 82, 23, 49, 77, 64, 76, 46,
        58, 72, 99, 39, 15, 69, 92, 38,  9, 19,  4, 20, 11, 80, 75, 43,  1,
        36, 29, 47, 26, 98, 18, 85, 17, 34, 60, 31, 95, 33,  8, 44, 55,  3,
        65,  6, 79, 56, 14, 45, 90, 59,  5, 27, 50,  2, 62,  7, 52, 22, 74,
        16,  0, 71, 87, 84, 97, 66, 70, 91, 88, 93, 83, 13, 63, 94, 28, 25,
        86, 24, 67, 42, 57, 61, 51, 96, 41, 48, 68, 37, 40, 12, 35],
       dtype=int32), 6496)

In [309]:
upper_bound_100x20=6286

(6496-upper_bound_100x20)*100/upper_bound_100x20

3.34075723830735

Starting from the NEH sequence, my algorithm converges to a state characterized by makespan= 6497, differing of 3.34% from the upper-bound solution.

Considering that the solution could be improved with more iterations (and more swaps, probably, in addition to much more computational power) the performance is actually satisfying, because the algorithm lowers the makespan best value significantly after some iterations.

### 500 jobs and 20 machines

In [9]:
#requiring the particular time matrix to the user
no_of_jobs=int(input('Please input the number of jobs: ') or 100) # default value is 20
no_of_machines=float(input('Please input the number of machines: ') or 20) # default value is 0.8
print("required computations on jobs: {}, machines: {}".format(no_of_jobs, no_of_machines) )

if no_of_jobs==20 and no_of_machines==5:
    df_tmp=pd.read_excel("/home/dario/Desktop/DecisionModel/Assignment_4/20x5_flowshop.xlsx",sheet_name="S1",index_col =[0]) #first of taill. 20x5 matrices
    upper_b=1278
elif no_of_jobs==100 and no_of_machines==20:
    df_tmp=pd.read_excel("/home/dario/Desktop/DecisionModel/Assignment_4/100x20_flowshop.xlsx",sheet_name="S1",index_col =[0]) #first of taill. 100x20 matrices
    upper_b=6286
elif no_of_jobs==500 and no_of_machines==20:
    df_tmp=pd.read_excel("/home/dario/Desktop/DecisionModel/Assignment_4/500x20_flowshop.xlsx",sheet_name="S1",index_col =[0]) #first of taill. 500x20 matrices
else:
    print("unsupported number of jobs or number of machines (not implemented)")

#print(pt_tmp.head())
m2=df_tmp.T.values #Transposing for personal preference and extracting the correspondent numpy 2D array
m2=m2.astype("int32") #ensuring it to be of type "int32", for better computational performance
print(m2.shape)

#m_list=df_tmp.T.values.tolist() #for a better iteration
#print(m_list)

Please input the number of jobs: 500
Please input the number of machines: 20
required computations on jobs: 500, machines: 20.0
(500, 20)


#### NEH solution

In [310]:
%time seq2, mks2=NEH(m2)
seq2,mks2

CPU times: user 34.5 s, sys: 3.08 ms, total: 34.5 s
Wall time: 34.5 s


(array([144, 451, 127, 287, 111, 294, 495, 406,  78, 430, 324, 272, 215,
        260, 158,  57, 368,  87, 474, 134, 180,  53, 275, 227,  90, 475,
        188,  15, 267,  42, 340,  52,  60, 109,  82, 337, 154, 410, 416,
        346, 191, 123, 449, 237, 450, 407, 292, 320, 339, 113,  44, 306,
        107, 323, 255, 378, 207, 224,  48, 305, 217, 397,  98, 162, 458,
        460,  18, 468, 281, 286, 327, 496, 143,   9, 353, 269,  89, 371,
        441, 233, 285, 119, 422, 322, 301, 192, 137, 424, 379,  67, 278,
         20,  21, 226, 326, 328, 100, 110, 349, 230, 244, 343, 334, 279,
        484,  38, 471, 225, 411, 216, 213, 126, 108, 241, 376, 498, 189,
        439, 122,  75, 288, 395, 152, 480, 303, 384, 448, 256,  77, 239,
        295, 138, 336, 473, 103, 297, 262, 387, 481,  63,   2,   3, 363,
        380, 273, 211,  46,  37,  86, 469, 208,   1, 431, 146,  22,  95,
        298, 235, 220, 102, 218, 313, 142, 497, 240, 247, 457, 203,  51,
         16,   4, 392, 263, 489, 231, 197,  55, 155

In [15]:
upper_bound_500x100=26189
#upper_bound_500x100=26040

(mks2-upper_bound_500x100)/upper_bound_500x100*100

1.8366489747603953

the NEH solution is 1.84% far from the upper-bound indicated in Taillard's benchmarks

Trying to improve the performance with the iterated local search, we obtain:

In [12]:
IteratedLocalSearch(seq2, m2, verbose=True,maxIterations=10, maxIterNoChange=10)

(array([346, 294, 406, 301, 324, 226, 430,  87, 260, 281, 474,   9, 267,
        109, 134, 340, 497,  53, 410, 127, 123, 450,  82, 191,  60, 287,
        337,  90, 275, 320, 395, 292, 215,  57, 162, 306, 339, 368, 107,
        207, 224, 180, 286,  52, 397, 217, 327,  98,  18, 496,  44, 269,
        475, 233, 119, 438, 322, 371,   0, 113, 323, 192, 154, 143, 144,
        441, 471, 495, 326, 349,  21,  89,  67, 498,  20, 422,  78, 343,
        230, 272, 241, 448, 328,  77,  75,  42, 279, 288, 103, 110, 216,
         38, 213, 225, 152, 295, 100, 262,  46, 387, 297, 239, 484, 158,
        244, 481, 189, 256, 242, 465, 235,  22,   1, 218, 208, 431, 102,
        458, 138, 108, 303, 380, 126, 384, 273,   2, 411, 278, 363, 392,
        220, 439, 336, 211,  37, 457, 473, 424, 379, 489, 460, 449, 353,
         55,  16, 263,   3, 188, 285,  48,   4,   6,  47,  96, 122,  15,
         95, 313, 155, 240, 298, 486, 156, 146, 245, 231, 184, 255, 182,
         58, 142, 480,  40, 468, 203, 166, 149, 163

In [17]:
upper_bound_500x100=26189
#upper_bound_500x100=26040

(26552-upper_bound_500x100)/upper_bound_500x100*100

1.3860781244033755

Due to quite limited computational resources, i could perform only 10 cycles of the ILS. 

However, after those iterations the gap between the upper bound reduced up to 1.39%

#### SA on 500x20

In [337]:
%time s, mks500= SimAnn(seq2, m2, n_iterations=10000000, alpha=0.99999,maxIterNoChange=1000000, t_ini=5,tmin=0.0001, n_swap=10)

mks500

Reached max number of iteration without improvment( 1000000) after 1000000 iterations
0.00022698829904838723
CPU times: user 10min 13s, sys: 60.2 ms, total: 10min 13s
Wall time: 10min 13s


26670

In [342]:
flow = FlowShopProb(seq2, m2)
flow.copy_strategy = "deepcopy"

#trying the auto strategy for having a better comparison with my implementation, considering comparable CPU time 
auto_schedule = flow.auto(minutes=10)  
flow.set_schedule(auto_schedule)
%time state, e = flow.anneal()

state, e

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01000      27042.00     0.03%     0.00%     0:08:12     0:00:00

CPU times: user 8min 12s, sys: 138 ms, total: 8min 12s
Wall time: 8min 12s


(array([144, 451, 127, 287, 111, 294, 495, 406,  78, 430, 324, 272, 215,
        260, 158,  57, 368,  87, 474, 134, 180,  53, 275, 227,  90, 475,
        188,  15, 267,  42, 340,  52,  60, 109,  82, 337, 154, 410, 416,
        346, 191, 123, 449, 237, 450, 407, 292, 320, 339, 113,  44, 306,
        107, 323, 255, 378, 207, 224,  48, 305, 217, 397,  98, 162, 458,
        460,  18, 468, 281, 286, 327, 496, 143,   9, 353, 269,  89, 371,
        441, 233, 285, 119, 422, 322, 301, 192, 137, 424, 379,  67, 278,
         20,  21, 226, 326, 328, 100, 110, 349, 230, 244, 343, 334, 279,
        484,  38, 471, 225, 411, 216, 213, 126, 108, 241, 376, 498, 189,
        439, 122,  75, 288, 395, 152, 480, 303, 384, 448, 256,  77, 239,
        295, 138, 336, 473, 103, 297, 262, 387, 481,  63,   2,   3, 363,
        380, 273, 211,  46,  37,  86, 469, 208,   1, 431, 146,  22,  95,
        298, 235, 220, 102, 218, 313, 142, 497, 240, 247, 457, 203,  51,
         16,   4, 392, 263, 489, 231, 197,  55, 155

The SA algorithm does not perform in such a situation of great number of jobs. It does not find any sequence better than the initial one, obtained with the NEH. 

One could increase the number of swap, and possibly the number of iteration, in order to gain better performances. However, this will require much more computational time.

However, the comparison with the library algorithm does not show any sign of improvement for this last attempt.

## Conclusions

The most performing algorithm seems to be the Annealing, if tested on a matrix up to 100x20. 

On the other side, the ILS seems a very stable process, giving a result constantly better than the NEH point of initialization.

This initialization with the NEH resulting sequence seems to be very profitable in terms of general performances.

The GA algorithm does not seem to perform well. It is particularly slow even on the 20x5 matrix we tested, and further tests on higher rank matrices would lead to huge CPU time requirements.


In general, further improving in performance can be obtained by executing more iterations.

The problem is that doing so would require a lot of CPU time, running this Pythonic/Cythonic implementation of the algorithm.

In order to reduce the required time, one should actually write the same algorithm down in a much lower level language, such as C/C++ or even better Fortran. 

I tried myself an approach by compiling some basic function (makespan and permutations) with Ft2py, however, mainly due to the lack of available (real!) time, i was not able to integrate the basic data structure within a numpy array (the most C-like looking structures in Py, at least up to my knowledge). 